# Анализ развития игровой индустрии в начале XXI века с акцентом на игры RPG

- Автор: Матвеева Анна
- Дата: Февраль 2025

# Цели и задачи проекта

Подготовка исходных данных для написания статьи о развитии индустрии игр в начале XXI века. Проведение исследовательской работы игровых платформ и объёмов продаж игр разных жанров, региональные предпочтения игроков (с акцентом на играх жанра RPG).

## Описание данных

Данные `/datasets/new_games.csv` содержат информацию о продажах игр разных жанров и платформ, а также пользовательские и экспертные оценки игр:  
- `Name` — название игры.  
- `Platform` — название платформы.  
- `Year of Release` — год выпуска игры.  
- `Genre` — жанр игры.  
- `NA sales` — продажи в Северной Америке (в миллионах проданных копий).  
- `EU sales` — продажи в Европе (в миллионах проданных копий).  
- `JP sales` — продажи в Японии (в миллионах проданных копий).  
- `Other sales` — продажи в других странах (в миллионах проданных копий).  
- `Critic Score` — оценка критиков (от 0 до 100).  
- `User Score` — оценка пользователей (от 0 до 10).  
- `Rating` — рейтинг организации ESRB (англ. Entertainment Software Rating Board).

## Содержимое проекта

---
1. Знакомство с данными.   
2. Проверка ошибок в данных и их предобработка. 
    - Названия, или метки, столбцов датафрейма.
    - Типы данных.
    - Наличие пропусков в данных.
    - Явные и неявные дубликаты в данных.
3. Фильтрация данных.  
4. Категоризация данных.

В ходе данного проекта планируется выполнить следующее:
- Проверить корректность данных и провести их предобработку.
- Отобрать данные по времени выхода игры за период с 2000 по 2013 год включительно.
- Категоризовать игры по оценкам пользователей и экспертов, выделив три категории:  
    - высокая оценка — с оценкой от 8 до 10 и от 80 до 100, включая правые границы интервалов.
    - средняя оценка — с оценкой от 3 до 8 и от 30 до 80, не включая правые границы интервалов.
    - низкая оценка — с оценкой от 0 до 3 и от 0 до 30, не включая правые границы интервалов.
- Выделить топ-7 платформ по количеству игр, выпущенных за весь требуемый период.  

Вышеперечисленные шаги позволят получить экспертам необходимый для написания статьи срез данных.

## 1. Загрузка данных и знакомство с ними

In [1]:
# Импортируем библиотеку pandas
import pandas as pd
import matplotlib.pyplot as plt

In [2]:
# Выгружаем данные из датасета new_games.csv в датафрейм games
games = pd.read_csv('https://code.s3.yandex.net/datasets/new_games.csv')

In [3]:
# Выводим информацию о датафрейме
games.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 16956 entries, 0 to 16955
Data columns (total 11 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   Name             16954 non-null  object 
 1   Platform         16956 non-null  object 
 2   Year of Release  16681 non-null  float64
 3   Genre            16954 non-null  object 
 4   NA sales         16956 non-null  float64
 5   EU sales         16956 non-null  object 
 6   JP sales         16956 non-null  object 
 7   Other sales      16956 non-null  float64
 8   Critic Score     8242 non-null   float64
 9   User Score       10152 non-null  object 
 10  Rating           10085 non-null  object 
dtypes: float64(4), object(7)
memory usage: 1.4+ MB


In [4]:
# Выводим первые строки датафрейма на экран
games.head(10)

,Name,Platform,Year of Release,Genre,NA sales,EU sales,JP sales,Other sales,Critic Score,User Score,Rating
0,Wii Sports,Wii,2006.0,Sports,41.36,28.96,3.77,8.45,76.0,8,E
1,Super Mario Bros.,NES,1985.0,Platform,29.08,3.58,6.81,0.77,NaN,NaN,NaN
2,Mario Kart Wii,Wii,2008.0,Racing,15.68,12.76,3.79,3.29,82.0,8.3,E
3,Wii Sports Resort,Wii,2009.0,Sports,15.61,10.93,3.28,2.95,80.0,8,E
4,Pokemon Red/Pokemon Blue,GB,1996.0,Role-Playing,11.27,8.89,10.22,1.00,NaN,NaN,NaN
5,Tetris,GB,1989.0,Puzzle,23.20,2.26,4.22,0.58,NaN,NaN,NaN
6,New Super Mario Bros.,DS,2006.0,Platform,11.28,9.14,6.5,2.88,89.0,8.5,E
7,Wii Play,Wii,2006.0,Misc,13.96,9.18,2.93,2.84,58.0,6.6,E
8,New Super Mario Bros. Wii,Wii,2009.0,Platform,14.44,6.94,4.7,2.24,87.0,8.4,E
9,Duck Hunt,NES,1984.0,Shooter,26.93,0.63,0.28,0.47,NaN,NaN,NaN


In [5]:
# Считаем пропуски в датафрейме
games.isna().sum()

Name                  2
Platform              0
Year of Release     275
Genre                 2
NA sales              0
EU sales              0
JP sales              0
Other sales           0
Critic Score       8714
User Score         6804
Rating             6871
dtype: int64

In [6]:
# Считаем % пропусков в датафрейме
round(games.isna().sum() / len(games)*100, 2)

Name                0.01
Platform            0.00
Year of Release     1.62
Genre               0.01
NA sales            0.00
EU sales            0.00
JP sales            0.00
Other sales         0.00
Critic Score       51.39
User Score         40.13
Rating             40.52
dtype: float64

In [7]:
# Выводим строки с пропусками
games[['Name','Year of Release','Genre','Critic Score','User Score','Rating']].isna().sum()

Name                  2
Year of Release     275
Genre                 2
Critic Score       8714
User Score         6804
Rating             6871
dtype: int64

**Первичные выводы.**  
Для анализа представлен датасет `new_games.csv`, который содержит 11 столбцов и 16956 строк. В нем представлена информация о продажах игр разных жанров и платформ, а также пользовательские и экспертные оценки игр.   
1. Проведен анализ типов данных и их корректность:  
    -	**Числовые значения с плавающей запятой (`float64`)**. 
    Четыре столбца представлены типом float64 (`Year of Release, NA sales, Other sales, Critic Score`).  
    Для трех столбцов  (`NA sales, Other sales, Critic Score`) это верное решение, так как сумма продаж в миллионах шт. включает дробные значения.  
    Для столбца `Year of Release` более уместен тип `int16`, т.к. год выпуска игры укладывается в ограничения данного типа данных.   
    •	**Строковые данные (`object`)**. Семь столбцов имеют данный тип данных:  
        `Name, Platform, Genre, Rating` содержат строковую информацию, что логично для текстовых данных. Здесь тип данных `object` подходит.  
        `EU sales, JP sales, User Score`  хранят информацию о суммах продаж и рейтингах игр, а значит существует возможность оптимизации путем корректировки типа на `float64` (Числовые значения с плавающей запятой).  
    •	**Булевые значения (`bool`)** в представленном датасете не требуются, т.к. отсутствуют столбцы, содержащие логическую информацию: `True` и `False`.  

После анализа типов данных видно, что половина столбцов корректно представлены. Для оптимизации можно использовать целочисленные типы с уменьшенной разрядностью и преобразовать строковые данные, содержащие числовые значения с запятой, в тип `float64`.


2. Наименования столбцов не соответствуют общепринятым стандартам **snake_case**. Необходима корректировка, поэтому заменим нижним подчеркиванием пробелы, используемые для разделения слов в названиях столбцов. Так же все буквы сделаем строчными с целью недопущения наличия дубликатов.

3. Датасет **содержит пропуски**, которые можно поделить на 2 категории:  
•	MCAR — пропуски совершенно случайны (`Name, Year of Release, Genre`);  
•	MAR — пропуски случайны, но объясняются наблюдаемыми данными (`Critic Score, User Score, Rating`);  
Первую категорию можно удалить, т.к. количество строк незначительное и не повлияет на итоговые результаты.
Вторую категорию удалять нельзя, при дальнейшем анализе можно будет выяснить причины. В данных ячейках будет установлен индикатор-"заглушка" для оперативного отфильтровывания данных строк.

## 2.  Проверка ошибок в данных и их предобработка


### 2.1. Названия, или метки, столбцов датафрейма

<font color='#777778'>- Выведите на экран названия всех столбцов датафрейма и проверьте их стиль написания.</font>  
<font color='#777778'>- Приведите все столбцы к стилю snake case. Названия должны быть в нижнем регистре, а вместо пробелов — подчёркивания.</font>

In [8]:
print('Наименования столбцов:')
games.columns.tolist()

Наименования столбцов:


['Name',
 'Platform',
 'Year of Release',
 'Genre',
 'NA sales',
 'EU sales',
 'JP sales',
 'Other sales',
 'Critic Score',
 'User Score',
 'Rating']

In [9]:
#Заменяем названия столбцов на более корректные snake_case
gm = games.copy()
gm.columns = gm.columns.str.replace(' ', '_').str.lower()
print('Наименования snake_case:')
gm.columns.tolist()

Наименования snake_case:


['name',
 'platform',
 'year_of_release',
 'genre',
 'na_sales',
 'eu_sales',
 'jp_sales',
 'other_sales',
 'critic_score',
 'user_score',
 'rating']

### 2.2. Типы данных

Как было сказано выше, типы данных не оптимальны по ряду столбцов, поэтому необходимо провести обработку пропусков, прежде, чем преобразовывать данные.

In [10]:
# Устанавливаем на пропущенные 275 позиций года выпуска нулевой год и меняем тип
print(gm['year_of_release'].min())
print(gm['year_of_release'].max())
gm['year_of_release'] = gm['year_of_release'].fillna(0).astype('int16')

1980.0
2016.0


In [11]:
# Удаляем строки с пропусками в наименовании и жанре, т.к. заполнить их нет закономерности, 
# а количество строк с пропусками ничтожно мало.
gm = gm.dropna(subset=['name'])
gm = gm.dropna(subset=['genre'])
gm.isna().sum()

name                  0
platform              0
year_of_release       0
genre                 0
na_sales              0
eu_sales              0
jp_sales              0
other_sales           0
critic_score       8712
user_score         6802
rating             6869
dtype: int64

In [12]:
# Преобразовываем строковый тип данных столбцов продажи в Европе и в Японии, 
# оценка пользователей и экспертов в числовые с плавающей запятой
gm['eu_sales'] = pd.to_numeric(gm['eu_sales'], errors = 'coerce' )
gm['jp_sales'] = pd.to_numeric(gm['jp_sales'], errors='coerce')
gm['critic_score'] = pd.to_numeric(gm['critic_score'], errors='coerce')
gm['user_score'] = pd.to_numeric(gm['user_score'], errors='coerce')

In [13]:
gm[gm.isna().any(axis=1)]  # Выводим строки где есть хоть одно пропущенное значение в каждой строке.

,name,platform,year_of_release,genre,na_sales,eu_sales,jp_sales,other_sales,critic_score,user_score,rating
1,Super Mario Bros.,NES,1985,Platform,29.08,3.58,6.81,0.77,NaN,NaN,NaN
4,Pokemon Red/Pokemon Blue,GB,1996,Role-Playing,11.27,8.89,10.22,1.00,NaN,NaN,NaN
5,Tetris,GB,1989,Puzzle,23.20,2.26,4.22,0.58,NaN,NaN,NaN
9,Duck Hunt,NES,1984,Shooter,26.93,0.63,0.28,0.47,NaN,NaN,NaN
10,Nintendogs,DS,2005,Simulation,9.05,10.95,1.93,2.74,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...
16951,Samurai Warriors: Sanada Maru,PS3,2016,Action,0.00,0.00,0.01,0.00,NaN,NaN,NaN
16952,LMA Manager 2007,X360,2006,Sports,0.00,0.01,0.00,0.00,NaN,NaN,NaN
16953,Haitaka no Psychedelica,PSV,2016,Adventure,0.00,0.00,0.01,0.00,NaN,NaN,NaN
16954,Spirits & Spells,GBA,2003,Platform,0.01,0.00,0.00,0.00,NaN,NaN,NaN


In [14]:
# Считаем возникшие пропуски в датафрейме
gm.isna().sum()

name                  0
platform              0
year_of_release       0
genre                 0
na_sales              0
eu_sales              6
jp_sales              4
other_sales           0
critic_score       8712
user_score         9266
rating             6869
dtype: int64

In [15]:
# Заменим пропуски в столбцах средним значением для каждой группы по платформе и году выпуска игры.
gm['jp_sales'] = gm['jp_sales'].fillna(gm.groupby(['platform', 'year_of_release'])['jp_sales'].transform('median'))
gm['eu_sales'] = gm['eu_sales'].fillna(gm.groupby(['platform', 'year_of_release'])['eu_sales'].transform('median'))                                      

### 2.3. Наличие пропусков в данных

In [16]:
# Считаем возникшие пропуски в датафрейме
gm.isna().sum()

name                  0
platform              0
year_of_release       0
genre                 0
na_sales              0
eu_sales              0
jp_sales              0
other_sales           0
critic_score       8712
user_score         9266
rating             6869
dtype: int64

In [17]:
# Считаем % пропусков в датафрейме
round(gm.isna().sum() / len(gm)*100, 2)

name                0.00
platform            0.00
year_of_release     0.00
genre               0.00
na_sales            0.00
eu_sales            0.00
jp_sales            0.00
other_sales         0.00
critic_score       51.39
user_score         54.65
rating             40.52
dtype: float64

In [18]:
# Считаем кол-во игр и их оценки/рейтинги в разрезе года выпуска
gm_group = gm.groupby('year_of_release').agg({
    'name':'count',
    'critic_score':'count',
    'user_score':'count',
    'rating':'count'
})
gm_group

,name,critic_score,user_score,rating
year_of_release,,,,
0,275,157,130,184
1980,9,0,0,0
1981,46,0,0,0
1982,37,0,0,0
1983,18,0,0,0
1984,14,0,0,0
1985,14,1,1,1
1986,22,0,0,0
1987,17,0,0,0


По данным выше видно, что наиболее полную оценку можно сделать по играм, вышедшим после 2000 года (как и указано в ТЗ). Игры более ранних периодов в большинстве своем не имеют рейтингов и оценок, т.к. игровая индустрия только начинала развиваться.

In [19]:
# Заменим все пустые значения индикаторами-"заглушками" для возможности дальнейшего анализа:
gm['critic_score'] = gm['critic_score'].fillna(-1) # оценка критиков (от 0 до 100)
gm['user_score'] = gm['user_score'].fillna(-1) # оценка пользователей (от 0 до 10)
gm['rating'] = gm['rating'].fillna('no_rating')

In [20]:
# Проверяем, имеется ли игра, которую не купили ни разу
# Фильтрация строк, содержащих пропуски (NaN) во всех четырех столбцах SALES
filtered_df = gm[gm['na_sales'].isna() & gm['eu_sales'].isna() & gm['jp_sales'].isna() & gm['other_sales'] ]
# Получение уникальных значений столбца 'Name'
unique_names = filtered_df['name'].unique()
print (f"Игры, не имеющие ни одной продажи: {sorted(unique_names)}")

Игры, не имеющие ни одной продажи: []


In [21]:
gm.head(10)

,name,platform,year_of_release,genre,na_sales,eu_sales,jp_sales,other_sales,critic_score,user_score,rating
0,Wii Sports,Wii,2006,Sports,41.36,28.96,3.77,8.45,76.0,8.0,E
1,Super Mario Bros.,NES,1985,Platform,29.08,3.58,6.81,0.77,-1.0,-1.0,no_rating
2,Mario Kart Wii,Wii,2008,Racing,15.68,12.76,3.79,3.29,82.0,8.3,E
3,Wii Sports Resort,Wii,2009,Sports,15.61,10.93,3.28,2.95,80.0,8.0,E
4,Pokemon Red/Pokemon Blue,GB,1996,Role-Playing,11.27,8.89,10.22,1.00,-1.0,-1.0,no_rating
5,Tetris,GB,1989,Puzzle,23.20,2.26,4.22,0.58,-1.0,-1.0,no_rating
6,New Super Mario Bros.,DS,2006,Platform,11.28,9.14,6.50,2.88,89.0,8.5,E
7,Wii Play,Wii,2006,Misc,13.96,9.18,2.93,2.84,58.0,6.6,E
8,New Super Mario Bros. Wii,Wii,2009,Platform,14.44,6.94,4.70,2.24,87.0,8.4,E
9,Duck Hunt,NES,1984,Shooter,26.93,0.63,0.28,0.47,-1.0,-1.0,no_rating


### 2.4. Явные и неявные дубликаты в данных

In [22]:
# Считаем дубликаты в категориальных данных (названия жанра, платформы, рейтинга и года выпуска).
count_dup1 = gm.duplicated(subset=['name','platform','year_of_release','genre','rating']).sum()
print(count_dup1)

183


In [23]:
# Считаем количество уникальных значений в столбцах
unique_count = gm[['name','platform','year_of_release','genre','rating']].nunique()
# Выводим результат
print(f'Количество уникальных значений: {unique_count}')

Количество уникальных значений: name               11559
platform              31
year_of_release       38
genre                 24
rating                 9
dtype: int64


In [24]:
# Выводим уникальные значения в столбцах
unique_count = gm['genre'].unique()
# Выводим результат
print(f'Уникальные значения жанра до изменения регистра: {unique_count} - дубли зафиксированы !')
unique_count = gm['rating'].unique()
# Выводим результат
print(f'Уникальные значения рейтинга до изменения регистра: {unique_count}')

Уникальные значения жанра до изменения регистра: ['Sports' 'Platform' 'Racing' 'Role-Playing' 'Puzzle' 'Misc' 'Shooter'
 'Simulation' 'Action' 'Fighting' 'Adventure' 'Strategy' 'MISC'
 'ROLE-PLAYING' 'RACING' 'ACTION' 'SHOOTER' 'FIGHTING' 'SPORTS' 'PLATFORM'
 'ADVENTURE' 'SIMULATION' 'PUZZLE' 'STRATEGY'] - дубли зафиксированы !
Уникальные значения рейтинга до изменения регистра: ['E' 'no_rating' 'M' 'T' 'E10+' 'K-A' 'AO' 'EC' 'RP']


In [25]:
# Приведем названий игр и жанров к нижнему регистру
gm['name'] = gm['name'].str.lower()
gm['genre'] = gm['genre'].str.lower()
# Приведем названий рейтинга к верхнему регистру
gm['rating'] = gm['rating'].str.upper()

In [26]:
# Считаем значения в нормализованных данных по ключевым столбцам (название жанра, платформы, рейтинг и год выпуска).
count_dup1 = gm.duplicated(subset=['name','platform','year_of_release','genre','rating']).sum()
print(count_dup1)

242


In [27]:
# Провка наличия явных дубликатов
gm.duplicated().sum()

241

В ходе проведения нормализации данных, по факту приведения к единому стилю, в датасете зафиксировано наличие явных дубликатов. Т.к. данные материалы содержат информацию о продажах (которая задвоилась), считаю корректным данные дубикаты удалить.

In [28]:
# Сохраняем количество строк до удаления дубликатов
initial_row_count = games.shape[0]
# Сортируем предобработанный датафрейм по всем столбцам
gm_sorted = gm.sort_values(by=list(gm.columns))
# Удаляем дубликаты
gm_no_duplicates = gm_sorted.drop_duplicates()
# Сохраняем количество строк после удаления дубликатов
final_row_count = gm_no_duplicates.shape[0]

# ЗНАЧЕНИЯ СКОРРЕКТИРОВАНы:
# Выводим результаты
print(f"Первоначальное количество строк до удаления  пропусков и дубликатов: {initial_row_count}")
print(f"Количество удаленных строк: {initial_row_count-final_row_count} шт., что составляет {round((100-final_row_count/initial_row_count*100),2)} %")
print(f"Количество строк после удаления пропусков и дубликатов: {final_row_count}")

Первоначальное количество строк до удаления  пропусков и дубликатов: 16956
Количество удаленных строк: 243 шт., что составляет 1.43 %
Количество строк после удаления пропусков и дубликатов: 16713


По факту проведения предобработки были преобразованы типы  данных, обработаны (преобразованы) пропуски, удалены некорректные данные, удалены явные дубликаты, а именно:  
1. Была проведена замена типов данных на более оптимальные:
- Для столбца `Year of Release` применен тип `int16`, т.к. год выпуска игры укладывается в ограничения данного типа данных.
- Для столбцов `EU sales, JP sales, User Score`проведена корректировки типа на `float64`, т.к. хранят информацию о суммах продаж и рейтингах игр. 
- В ходе корретировки зафиксировано появление в столбцах некорректных значений (обработаны при анализе пропусков).
2. Наименования столбцов не соответствовали общепринятым стандартам **snake_case**. Проведена корректировка регистра с целью исключения дубликатов и замена пробелов на нижнее подчеркивание.
3. Датасет содержал пропуски. 
- 2 строки были удалены (пропуск в названии и жанре), т.к. количество строк незначительное и не повлияет на итоговые результаты. 
- В столбцах `eu_sales, jp_sales` пропуски заменены на медианные значения в группах `по platform, year_of_release`. 
- Пропуски в количестве 275 строк в столбце `Year of Release` заполнены индикатором-"заглушкой" (год = 0). При этом, данное количество так же является не значительным (<2%), не попадает в итоговый датасет и никак не влияет на итоговые результаты. 
- Проанализированы столбцы с высоким % пропусков (`critic_score` - 51.39 %, `user_score` - 54.65 %, `rating` - 40.52%): основная масса пропусков присутствует в играх до 2000 года выпуска. Видимо, до этого года оценки и рейтинги практически не использовались. В столбцах установлены индикаторы-"заглушки".
4. В ходе проработки явных и не явных **дубликатов** количество строк с 16 959 шт. сократилось на 243 шт. (равно 1.43 %) и составило 16 713 шт.

---

## 3. Фильтрация данных

In [29]:
# Фильтруем данные за 2000-2013 годы включительно (согласно ТЗ)
# gm_actual = gm_no_duplicates.copy()
gm_actual = gm_no_duplicates[gm_no_duplicates['year_of_release'].between(2000, 2013, inclusive=True)].reset_index()
gm_actual

,index,name,platform,year_of_release,genre,na_sales,eu_sales,jp_sales,other_sales,critic_score,user_score,rating
0,3394,frozen: olaf's quest,3DS,2013,platform,0.27,0.27,0.00,0.05,-1.0,-1.0,NO_RATING
1,3906,frozen: olaf's quest,DS,2013,platform,0.21,0.26,0.00,0.04,-1.0,-1.0,NO_RATING
2,2478,tales of xillia 2,PS3,2012,role-playing,0.20,0.12,0.45,0.07,71.0,7.9,T
3,8460,.hack//g.u. vol.1//rebirth,PS2,2006,role-playing,0.00,0.00,0.17,0.00,-1.0,-1.0,NO_RATING
4,7182,.hack//g.u. vol.2//reminisce,PS2,2006,role-playing,0.11,0.09,0.00,0.03,-1.0,-1.0,NO_RATING
...,...,...,...,...,...,...,...,...,...,...,...,...
12776,7233,zumba fitness: world party,Wii,2013,misc,0.11,0.10,0.00,0.02,-1.0,-1.0,E
12777,6970,zumba fitness: world party,XOne,2013,misc,0.17,0.05,0.00,0.02,73.0,6.2,E
12778,15739,zwei!!,PSP,2008,role-playing,0.00,0.00,0.02,0.00,-1.0,-1.0,NO_RATING
12779,13138,zyuden sentai kyoryuger: game de gaburincho!!,3DS,2013,action,0.00,0.00,0.05,0.00,-1.0,-1.0,NO_RATING


---

## 4. Категоризация данных

In [30]:
# Функция разбиения на группы pd.cut
def critic_categorize_groups(row):
    return pd.cut([row['critic_score']], bins=[-1, 0, 30, 80, 100], labels=['нет оценки', 'низкая оценка', 'средняя оценка', 'высокая оценка'], right=False)[0]

# Применяем функцию с группами методом apply
gm_actual['critic_score_groups'] = gm_actual.apply(critic_categorize_groups, axis=1)

#gm_actual

In [31]:
# Функция разбиения на группы if_elif (очень быстро)
def user_category_groups(row):
    if row < 0:
        return 'нет оценки'
    elif 0 <= row < 3:
        return 'низкая оценка'
    elif 3 <= row < 8:
        return 'средняя оценка'
    elif 8 <= row <= 10:
        return 'высокая оценка'
    else:
        return None

# Применяем функцию методом .apply() к столбцу 'critic_score'
gm_actual['user_score_groups'] = gm_actual['user_score'].apply(user_category_groups)

#gm_actual

In [32]:
gm_actual.head(10)

,index,name,platform,year_of_release,genre,na_sales,eu_sales,jp_sales,other_sales,critic_score,user_score,rating,critic_score_groups,user_score_groups
0,3394,frozen: olaf's quest,3DS,2013,platform,0.27,0.27,0.00,0.05,-1.0,-1.0,NO_RATING,нет оценки,нет оценки
1,3906,frozen: olaf's quest,DS,2013,platform,0.21,0.26,0.00,0.04,-1.0,-1.0,NO_RATING,нет оценки,нет оценки
2,2478,tales of xillia 2,PS3,2012,role-playing,0.20,0.12,0.45,0.07,71.0,7.9,T,средняя оценка,средняя оценка
3,8460,.hack//g.u. vol.1//rebirth,PS2,2006,role-playing,0.00,0.00,0.17,0.00,-1.0,-1.0,NO_RATING,нет оценки,нет оценки
4,7182,.hack//g.u. vol.2//reminisce,PS2,2006,role-playing,0.11,0.09,0.00,0.03,-1.0,-1.0,NO_RATING,нет оценки,нет оценки
5,8719,.hack//g.u. vol.2//reminisce (jp sales),PS2,2006,role-playing,0.00,0.00,0.16,0.00,-1.0,-1.0,NO_RATING,нет оценки,нет оценки
6,8410,.hack//g.u. vol.3//redemption,PS2,2007,role-playing,0.00,0.00,0.17,0.00,-1.0,-1.0,NO_RATING,нет оценки,нет оценки
7,1575,.hack//infection part 1,PS2,2002,role-playing,0.49,0.38,0.26,0.13,75.0,8.5,T,средняя оценка,высокая оценка
8,9189,.hack//link,PSP,2010,role-playing,0.00,0.00,0.14,0.00,-1.0,-1.0,NO_RATING,нет оценки,нет оценки
9,3023,.hack//mutation part 2,PS2,2002,role-playing,0.23,0.18,0.20,0.06,76.0,8.9,T,средняя оценка,высокая оценка


In [33]:
# Группируем данные для подсчета количества игр в каждой категории
grouped_user = gm_actual.groupby('user_score_groups')['name'].count().reset_index(name='grouped_game')
grouped_critic = gm_actual.groupby('critic_score_groups')['name'].count().reset_index(name='groups_game')

# Объединяем данные в одну таблицу:
result_gm = pd.merge(grouped_user, grouped_critic, left_on='user_score_groups', right_on='critic_score_groups')
result_gm

,user_score_groups,grouped_game,critic_score_groups,groups_game
0,высокая оценка,2286,высокая оценка,1692
1,нет оценки,6298,нет оценки,5612
2,низкая оценка,116,низкая оценка,55
3,средняя оценка,4081,средняя оценка,5422


In [34]:
# Выделяем топ-7 платформ по количеству игр.
# Считаем количество игр для каждой платформы:
count_platform = gm_actual.groupby('platform')['name'].count().reset_index(name='count_game')
# Сортируем платформы по количеству игр в порядке убывания:
top_7 = count_platform.sort_values(by='count_game', ascending=False).reset_index(drop=True).head(7)
top_7

,platform,count_game
0,PS2,2127
1,DS,2120
2,Wii,1275
3,PSP,1180
4,X360,1121
5,PS3,1087
6,GBA,811


---

## 5. Итоговый вывод

<font color='#777778'>В конце напишите основной вывод и отразите, какую работу проделали. Не забудьте указать описание среза данных и новых полей, которые добавили в исходный датасет.</font>

Для анализа был представлен датасет `new_games.csv` из 11 столбцов и 16956 строк. В нем представлена информация о продажах игр разных жанров и платформ, а также пользовательские и экспертные оценки игр. Для корректного анализа была проведена предобработка данных.    
1. Была проведена замена типов данных на более оптимальные:  
    •	**Числовые значения с плавающей запятой (`float64`)**. 
    Четыре столбца представлены типом float64 (`Year of Release, NA sales, Other sales, Critic Score`). Для трех столбцов  (`NA sales, Other sales, Critic Score`) это верное решение, так как сумма продаж в миллионах шт. включает дробные значения. Для столбца `Year of Release` более уместен тип `int16`, т.к. год выпуска игры укладывается в ограничения данного типа данных.   
    •	**Строковые данные (`object`)**. Семь столбцов имеют данный тип данных. По трем столбцам `Name, Platform, Genre, Rating`, содержащим строковую информацию, тип данных `object` подходит. `EU sales, JP sales, User Score`  хранят информацию о суммах продаж и рейтингах игр -> проведена корректировки типа на `float64` (Числовые значения с плавающей запятой). 
    
    В ходе корретировки зафиксировано появление в столбцах некорректных значений (обработаны при анализе пропусков).  
    
    
2. Наименования столбцов не соответствовали общепринятым стандартам **snake_case**. Проведена корректировка регистра с целью исключения дубликатов и замена пробелов на нижнее подчеркивание.


3. Датасет содержал **пропуски**: 
- 2 строки были **удалены** (пропуск в названии и жанре), т.к. количество строк незначительное и не повлияет на итоговые результаты. 
- В столбцах `eu_sales, jp_sales` пропуски **заменены на медианные значения** в группах по `platform, year_of_release`. 
- Пропуски в количестве 275 строк в столбце `Year of Release` **заполнены** индикатором-"заглушкой" (год = 0). При этом, данное количество так же является не значительным (<2%), не попадает в итоговый датасет и никак не влияет на итоговые результаты. 
- Проанализированы столбцы с высоким % пропусков (`critic_score` - 51.39 %, `user_score` - 54.65 %, `rating` - 40.52%): основная масса пропусков присутствует в играх до 2000 года выпуска. Видимо, до этого года оценки и рейтинги практически не использовались. В столбцах **установлены индикаторы**-"заглушки". Поскольку в ТЗ определен период анализа (2000-2013), наличие пропусков не оказывает существенного влияния на итоговые выводы.

4. В ходе проработки явных и не явных **дубликатов** количество строк с 16 959 шт. сократилось на 243 шт. (равно 1.43 %) и составило 16 713 шт.


5. Поскольку перед командой стоит задача изучить историю продаж игр в начале XXI века, новый срез данных **за период с 2000 по 2013 год** был сохранен в отдельном датафрейме. Количество строк сократилось до 12 781 шт.


6. Проведена **категоризация** игр на основании оценок пользователей и критиков. Выделены **топ-7** платформ по количеству игр, выпущенных за период 2000-2013. Строки с отсутствующими оценками (с "заглушками") вынесены в отдельную группу "Нет оценки". 

Итогом анализа стали данные о топ-7 наиболее востребованных платформ, на которые необходимо ориентироваться при привлечении новой аудитории. Так же видно, что большая часть игр характеризуется (как пользователями, так и критиками) как средняя. Низкого качества игр достаточно мало. Разместив в статье материалы о наиболее востребованных высокорейтинговых играх, возможно, удастся привлечь новую аудиторию любителей старых игр, и заодно заинтересовать их «Секретами Темнолесья».

При том, что по ходу выполнения проекта, были выполнены все рекомендации и соблюдена последовательность действий, для себя я не увидела ответов на первично поставленные вопросы.  

Поэтому, для анализа объёмов продаж игр разных жанров и возможности рассмотрения региональных предпочтений игроков (считаю это требованием ТЗ), **были выполнены # ДОП.АНАЛИЗ 1 и # ДОП.АНАЛИЗ 2.** Данные материалы позволяют изучить общую востребованность игры во всех регионах продаж.  
Так же, с целью акцентирования в статье игр жанра RPG, имеется возможность проведения сравнительного анализа интересующего жанра.  

**<div class="alert-warning"> ДОП.АНАЛИЗ 2 показывает, что количество игр жанра RPG находится только на 4ом месте от общего кол-ва игр, а по количеству проданных копий (в миллионах шт.) на пятом.</div>** 

Возможно, доп.срезы помогут коллегам выпустить достойную статью на известном IT-ресурсе, что позволит 
привлечь новую аудиторию любителей старых игр, и заодно заинтересовать их «Секретами Темнолесья».

In [35]:
# ДОП.АНАЛИЗ 1
# Суммируем все продажи по регионам в разрезе каждой игры
sum_columns = gm_actual[['eu_sales', 'jp_sales', 'other_sales', 'na_sales']].sum(axis=1)
# Присваивание новых значений в новый столбец 'total_sum'
gm_actual['total_sales'] = sum_columns
# Сортировка DataFrame по убыванию значений в столбце 'total_sum'
gm_actual.sort_values(by='total_sales', ascending=False, inplace=True)

gm_actual.head(10)

,index,name,platform,year_of_release,genre,na_sales,eu_sales,jp_sales,other_sales,critic_score,user_score,rating,critic_score_groups,user_score_groups,total_sales
12233,0,wii sports,Wii,2006,sports,41.36,28.96,3.77,8.45,76.0,8.0,E,средняя оценка,высокая оценка,82.54
6063,2,mario kart wii,Wii,2008,racing,15.68,12.76,3.79,3.29,82.0,8.3,E,высокая оценка,высокая оценка,35.52
12234,3,wii sports resort,Wii,2009,sports,15.61,10.93,3.28,2.95,80.0,8.0,E,высокая оценка,высокая оценка,32.77
7435,6,new super mario bros.,DS,2006,platform,11.28,9.14,6.50,2.88,89.0,8.5,E,высокая оценка,высокая оценка,29.80
12231,7,wii play,Wii,2006,misc,13.96,9.18,2.93,2.84,58.0,6.6,E,средняя оценка,средняя оценка,28.91
7438,8,new super mario bros. wii,Wii,2009,platform,14.44,6.94,4.70,2.24,87.0,8.4,E,высокая оценка,высокая оценка,28.32
7622,10,nintendogs,DS,2005,simulation,9.05,10.95,1.93,2.74,-1.0,-1.0,NO_RATING,нет оценки,нет оценки,24.67
6062,11,mario kart ds,DS,2005,racing,9.71,7.47,4.13,1.90,91.0,8.6,E,высокая оценка,высокая оценка,23.21
12225,13,wii fit,Wii,2007,sports,8.92,8.03,3.60,2.15,80.0,7.7,E,высокая оценка,средняя оценка,22.70
5323,14,kinect adventures!,X360,2010,misc,15.00,4.89,0.24,1.69,61.0,6.3,E,средняя оценка,средняя оценка,21.82


In [36]:
# ДОП.АНАЛИЗ 2
# Считаем количество игр для каждого жанра, а так же сумму продаж всего и по регионам.
# Группировка по столбцу 'genre' с суммой 'total_sales'
grouped = gm_actual.groupby('genre').agg({
    'name': 'count',
    'total_sales': 'sum',
    'na_sales': 'sum',
    'eu_sales': 'sum',
    'jp_sales': 'sum',
    'other_sales': 'sum'
})
grouped = grouped.sort_values(by='name', ascending=False)
grouped

# Сортировка по количеству названий в порядке убывания
grouped = grouped.sort_values(by='name', ascending=False)
grouped

,name,total_sales,na_sales,eu_sales,jp_sales,other_sales
genre,,,,,,
action,2460,1324.73,679.74,389.85,102.63,152.51
sports,1818,1021.44,543.26,293.85,72.62,111.71
misc,1484,680.30,350.22,183.35,79.82,66.91
role-playing,1079,629.78,248.83,127.52,210.55,42.88
adventure,1009,166.19,76.49,43.01,33.74,12.95
shooter,1001,741.76,416.21,228.50,18.41,78.64
racing,966,533.54,263.57,177.89,25.98,66.10
simulation,724,322.28,159.54,97.40,37.85,27.49
platform,682,475.82,250.76,132.73,52.54,39.79
